In [1]:
#Import Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

In [2]:
#Launch Splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Pulling Data from Race Results Page

In [3]:
#Visit Race Results Page 
url = "https://rankings.usatriathlon.org/Event/ViewEvent/301770/1"
browser.visit(url)

In [4]:
#we want the HTML from the page our browser is currently on
html = browser.html


#soupify (parse) the html so we can read it
soup = bs(html, 'html.parser')

In [5]:
#Find the data - in this case where the role = gridcell
athlete_data = soup.find_all(attrs={"role": "gridcell"})
athlete_data

[]

In [6]:
#Extract the data and put it into a DataFrame 
count = 0
fullTable = []
currow = []

for cell in athlete_data:
            text = cell.find("div").text
            currow.append(text)
            count += 1
            if count == 9:
                fullTable.append(currow)
                currow = []
                count = 0
df = pd.DataFrame(fullTable)

In [7]:
df.head()

""


### Pulling URLS from Main Results Page

In [8]:
#Visit the main page 
url2 = "https://rankings.usatriathlon.org/Event/Events"
browser.visit(url2)

In [9]:
#Grab the HTML and soupify 
html2 = browser.html
soup2= bs(html2, 'html.parser')

In [10]:
#Bring back the URLS and race namesfor each race page 
links = soup2.find_all("a")
link_list = []
race_name = []
for anchor in links:
    links = anchor["href"]
    names = anchor.text
    
    link_list.append(links)
    race_name.append(names)

In [11]:
soup2.find_all("a")[4].text

'Bartow Blarney Triathlon'

In [12]:
#Create a DataFrame to hold the Race Name and the URLS - we'll iterate through this to extract data from each page
race_main = pd.DataFrame()
race_main['Race'] = race_name
race_main['link'] = ['https://rankings.usatriathlon.org'+ link for link in link_list]
#We'll be able to use try/except to filter out the pages that aren't actually race results

In [13]:
race_main

,Race,link
0,\n,https://rankings.usatriathlon.orghttp://usatri...
1,Race Results,https://rankings.usatriathlon.org/Event/Events
2,National Rankings,https://rankings.usatriathlon.org/Rankings/Nat...
3,Athlete Results,https://rankings.usatriathlon.org/RaceResult/A...
4,Bartow Blarney Triathlon,https://rankings.usatriathlon.org/Event/ViewEv...
5,College of the Florida Keys Sprint Triathlon/B...,https://rankings.usatriathlon.org/Event/ViewEv...
6,Florida Ultra Week,https://rankings.usatriathlon.org/Event/ViewEv...
7,UCSB Kendra's Race Triathlon,https://rankings.usatriathlon.org/Event/ViewEv...
8,Las Olas Fort Lauderdale Triathlon,https://rankings.usatriathlon.org/Event/ViewEv...
9,Pasadena Triathlon 2020,https://rankings.usatriathlon.org/Event/ViewEv...


### Put It All Together

In [14]:
for index,row in race_main.iterrows(): 
    url3 = row.link
    if '/ViewEvent' in url3:
        race = row.Race
        try:
            browser.visit(url3)
            html3 = browser.html
            soup3 = bs(html3, 'html.parser')
            print(race)
        except Exception as e: 
            print(e)

Bartow Blarney Triathlon
College of the Florida Keys Sprint Triathlon/Biathlon
Florida Ultra Week
UCSB Kendra's Race Triathlon
Las Olas Fort Lauderdale Triathlon
Pasadena Triathlon 2020
Clermont Draft Legal Challenge 
Tri WashU 
Stanford Treeathlon
3rd Annual Tri-Family Racing Lake Pleasant 
Desert Triathlon
Iceman Triathlon
Shelbyville Triathlon Series Race #1
Huntington's Disease Triathlon Sprint and Olympic - Racing 4 Research To Find a Cure
Chasing Caterpillars Triathlon 2020
Tritonman 2020
Ultraman Florida Triathlon
Winter Wonderland Triathlon
Tri-owa Indoor Triathlon
HITS Triathlon Series: Sarasota, FL


In [17]:
# create an empty df
USAT_data = pd.DataFrame()

#for row in link_data...
for index,row in race_main.iterrows():
    #set the url as the link in the dataframe
    url3 = row.link
    #if the url is a race results page (as opposed to a FB link or a URL fragment) then...
    if '/ViewEvent' in url3:
        #set the race variable as the name of the Race
        race = row.Race
        try:
            #visit the browser, grab the html, and soupify it
            browser.visit(url3)
            html3 = browser.html
            soup3 = bs(html3, 'html.parser')
            
            #find the athlete data
            triathlete_data = soup3.find_all(attrs={"role": "gridcell"})
            
            #get each row of data and store in df 
            count = 0
            fullTable = []
            currow = []
            for cell in triathlete_data:
                text = cell.find("div").text
                currow.append(text)
                count += 1
                if count == 9:
                    fullTable.append(currow)
                    currow = []
                    count = 0
            df = pd.DataFrame(fullTable)
            
            #add in a column with the race
            df['Race'] = race
            
            #add to the empty df
            USAT_data = USAT_data.append(df)
            
        #if it isn't successful, bring back the name of the race and the error
        except Exception as e: 
            print(race)
            print(e)

College of the Florida Keys Sprint Triathlon/Biathlon
'NoneType' object has no attribute 'text'
Florida Ultra Week
'NoneType' object has no attribute 'text'


In [18]:
USAT_data

,Race,0,1,2,3,4,5,6,7,8
0,3rd Annual Tri-Family Racing Lake Pleasant,1,CHARLIE,BOEYINK,M,47,AZ,Age Group,02:30:53.920,92.145
1,3rd Annual Tri-Family Racing Lake Pleasant,2,WILLIAM,OWSLEY,M,53,AZ,Age Group,02:31:38.470,91.694
2,3rd Annual Tri-Family Racing Lake Pleasant,3,ARA,PARSEGHIAN,M,36,AZ,Age Group,02:31:46.210,91.616
3,3rd Annual Tri-Family Racing Lake Pleasant,4,ANDREW,MALLY,M,40,AZ,Age Group,02:36:47.420,88.682
4,3rd Annual Tri-Family Racing Lake Pleasant,5,JAMES,MAMARIL-DAVIS,M,25,AZ,Age Group,02:43:00.730,85.298
5,3rd Annual Tri-Family Racing Lake Pleasant,6,CHARLIE,REED,M,25,MI,Age Group,02:45:11.390,84.173
6,3rd Annual Tri-Family Racing Lake Pleasant,7,SAM,GADZICHOWSKI,M,41,AZ,Age Group,02:46:11.350,83.667
7,3rd Annual Tri-Family Racing Lake Pleasant,8,DAVID,EDWARDS,M,31,AZ,Age Group,02:49:05.370,82.232
8,3rd Annual Tri-Family Racing Lake Pleasant,9,SHANNON,TREEN,F,38,AZ,Age Group,02:54:45.560,87.52
9,3rd Annual Tri-Family Racing Lake Pleasant,10,ANDREW,ROERICK,M,24,NM,Age Group,02:55:26.660,79.253
